# Deriving the formula for sample size in the case of the difference between two sample means


1. By the Central Limit Theorem, when $n$ is large the sample mean $\bar{X}$ is distributed as $N(\mu, \frac{\sigma^2)}{n}$, with $\mu$ as the population mean.
2. By the properties of normal distributions (i.e. variances are summed) the distribution of $D$, the difference between two sample means is $N(\mu_1 - \mu_2, \frac{\sigma_1^2}{n_1} + \frac{\sigma_2^2}{n_2})$
3. If we assume equally sized samples from populations with identical variance then this reduces to $N(\mu_1 - \mu_2, \frac{2 \sigma^2}{n})$
4. Let $\phi$ be the inverse cumulative distribution function of a standardised normal distribution. Then the inverse cdf for D for a given probability $x$ is $\phi(x) \frac{\sqrt{2} \sigma}{\sqrt{n}}$

To derive the formula for sample size we need to find a way of relating power, samples size, minimum detectable difference and the variance of the population. We do this as follows.

1. If $\alpha$ is the probability of a type I error then (assuming a two tailed test) the upper cut off point for the right tail rejection region is $ u = \phi(1-\frac{\alpha}{2}) \frac{\sqrt{2} \sigma}{\sqrt{n}}$
2. Assume we were carrying out a test of the null hypothesis that the difference between the two population means is zero and assume that in actual fact that the difference is equal to the minimum difference $m$ and assume this difference is positive (there is an equivalent deriviation if it is negative.). Then the upper cut off point $u$ is equivalent to $\phi(1-\rho) \frac{\sqrt{2} \sigma}{\sqrt{n}} + m$ where $\rho$ is the power of the test.
3. Now we can create the following equality and solve it (using the fact that $\phi(1-\rho) = -\phi(\rho)$)

$$ \phi(1-\frac{\alpha}{2}) \frac{\sqrt{2} \sigma}{\sqrt{n}} = \phi(1-\rho) \frac{\sqrt{2} \sigma}{\sqrt{n}} + m $$
$$ \phi(1-\frac{\alpha}{2}) \frac{\sqrt{2} \sigma}{\sqrt{n}} - \phi(1-\rho) \frac{\sqrt{2} \sigma}{\sqrt{n}} =  m $$
$$ \phi(1-\frac{\alpha}{2}) \frac{\sqrt{2} \sigma}{\sqrt{n}} + \phi(\rho) \frac{\sqrt{2} \sigma}{\sqrt{n}} =  m $$
$$ \frac{\sqrt{2} \sigma}{\sqrt{n}}(\phi(1-\frac{\alpha}{2})  + \phi(\rho)) =  m $$
$$ \frac{\sqrt{2} \sigma}{m}(\phi(1-\frac{\alpha}{2})  + \phi(\rho)) =  \sqrt{n} $$
$$ n = \frac{2 \sigma^2}{m^2}(\phi(1-\frac{\alpha}{2})  + \phi(\rho))^2  $$

Now as I am paranoid I will check this using Monte Carlo simulation

## Monte Carlo Simulation to double check the sample size formula for the difference between two means

First let's calculate the sample size for some values.

In [5]:
import numpy as np, math
from scipy import stats
from scipy.stats import norm

alpha = 0.05
power = 0.8
st_dev = 270.11
min_effect  = 0.1

#Get the inverse cdfs
z_alpha = stats.norm.ppf(1-alpha/2)
z_beta = stats.norm.ppf(power)


sample_size = int(round((z_alpha + z_beta)**2 * 2*st_dev**2 / min_effect**2))

print(sample_size)

114529930


Now we simulate how many type I and type II errors we would get with this sample size.

In [16]:
mu_1 = 0
mu_2 = mu_1 + min_effect

type_1_errors = 0
type_2_errors = 0
upper_rejection_region = norm.ppf(1-alpha/2)*math.sqrt(2)*st_dev/math.sqrt(sample_size)
lower_rejection_region = norm.ppf(alpha/2)*math.sqrt(2)*st_dev/math.sqrt(sample_size)

for i in range(10000):
    
    #Simulate the difference if the minimum difference was in effect
    
    population_1 = np.random.normal(loc=mu_1, scale = st_dev, size = sample_size)
    population_2 = np.random.normal(loc=mu_2, scale = st_dev, size = sample_size)
    
    sample_mean_1 = np.mean(population_1)
    sample_mean_2 = np.mean(population_2)
    
    diff_between_means = sample_mean_1 - sample_mean_2
    
    #Now if the difference comes between the lower and upper cut offs the test will be such that
    #null hypothesis is NOT rejected, despite the existence of a difference and we will get a type II error
    if diff_between_means < upper_rejection_region and diff_between_means > lower_rejection_region:
        type_2_errors = type_2_errors + 1
        
    
    #Simulate the difference if the null hypothesis was true
    
    population_1 = np.random.normal(loc=mu_1, scale = st_dev, size = sample_size)
    population_2 = np.random.normal(loc=mu_1, scale = st_dev, size = sample_size)
    
    sample_mean_1 = np.mean(population_1)
    sample_mean_2 = np.mean(population_2)
    
    diff_between_means = sample_mean_1 - sample_mean_2
    
    #Now if the difference falls in the rejection region the null hypothesis would be rejected, despite the fact
    #that it is true and we get a Type I error
    if diff_between_means > upper_rejection_region or diff_between_means < lower_rejection_region:
        type_1_errors = type_1_errors + 1 

In [17]:
print("Type I errors: %s" % (type_1_errors/10000))
print("Type II errors: %s" % (type_2_errors/10000))

Type I errors: 0.1049
Type II errors: 0.0477


Now experiment with the inputs